In [3]:
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any

import requests

In [7]:
HOST = 'localhost:5000'
URI = f'http://{HOST}/api/v1/generate'

class AlpacaLLM(LLM):
    
    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        if isinstance(stop, list):
            stop = stop + ["\n###","\nObservation:", "\nObservations:"]

        response = requests.post(
            URI,
            json={
                "prompt": prompt,
                "temperature": 0.7,
                "max_new_tokens": 256,
                "early_stopping": True,
                "stopping_strings": stop,
                'do_sample': True,
                'top_p': 0.1,
                'typical_p': 1,
                'repetition_penalty': 1.18,
                'top_k': 40,
                'min_length': 0,
                'no_repeat_ngram_size': 0,
                'num_beams': 1,
                'penalty_alpha': 0,
                'length_penalty': 1,
                'seed': -1,
                'add_bos_token': True,
                'truncation_length': 2048,
                'ban_eos_token': False,
                'skip_special_tokens': True,
            },
        )
        response.raise_for_status()
        return response.json()['results'][0]['text']

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {}

In [8]:
llm = AlpacaLLM()

In [10]:

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.utilities import WikipediaAPIWrapper
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union, Any, Optional, Type
from langchain.schema import AgentAction, AgentFinish
import re
import os
from langchain import PromptTemplate
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from langchain.utilities import GoogleSerperAPIWrapper

In [11]:
template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the following questions as best you can. You have access to the following tools:

Google Search: A wrapper around Google Search. Useful for when you need to answer questions about current events. The input is the question to search relavant information.

Strictly use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Google Search]
Action Input: the input to the action, should be a question.
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

For examples:
Question: How old is CEO of Microsoft wife?
Thought: First, I need to find who is the CEO of Microsoft.
Action: Google Search
Action Input: Who is the CEO of Microsoft?
Observation: Satya Nadella is the CEO of Microsoft.
Thought: Now, I should find out Satya Nadella's wife.
Action: Google Search
Action Input: Who is Satya Nadella's wife?
Observation: Satya Nadella's wife's name is Anupama Nadella.
Thought: Then, I need to check Anupama Nadella's age.
Action: Google Search
Action Input: How old is Anupama Nadella?
Observation: Anupama Nadella's age is 50.
Thought: I now know the final answer.
Final Answer: Anupama Nadella is 50 years old.

### Input:
{input}

### Response:
{agent_scratchpad}"""


In [19]:
temp_Ins = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Question: {thought}
Query: {query}
Observation: {observation}

### Input:
Make a short summary of useful information from the result observation that is related to the question.

### Response:"""

prompt_Ins = PromptTemplate(
    input_variables=["thought", "query", "observation"],
    template=temp_Ins,
)

In [67]:
class CustomPromptTemplate(StringPromptTemplate):

    input_variables: List[str]
    """A list of the names of the variables the prompt template expects."""

    template: str
    """The prompt template."""

    template_format: str = "f-string"
    """The format of the prompt template. Options are: 'f-string', 'jinja2'."""

    validate_template: bool = False
    """Whether or not to try validating the template."""

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way

        print(f"format. kwargs: {kwargs} !!!!")
        
        intermediate_steps = kwargs.pop("intermediate_steps")
        
        thoughts = ""
        # Refine the observation
        if len(intermediate_steps) > 0:
            # regex = r"Thought\s*\d*\s*:(.*?)\nAction\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)\nObservation"
            regex = r"Thought: (.+)\nAction: (.+)\nAction Input: (.+)"
            text_match = intermediate_steps[-1][0].log

            print(f"format. text_match: {text_match}  !!!!")
            
            if len(intermediate_steps) > 1:
                text_match = 'Thought: ' + text_match
            match = re.search(regex, text_match, re.DOTALL) 

            print(f"match: {match.group(1)}, {match.group(3)} !!!")
            
            my_list = list(intermediate_steps[-1])
            p_INS_temp = prompt_Ins.format(thought=match.group(1).strip(), query=match.group(3).strip(), observation=my_list[1])
            my_list[1] = llm(p_INS_temp)
            my_tuple = tuple(my_list)            
            intermediate_steps[-1] = my_tuple
            
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f" {observation}\nThought:"
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        
        return self.template.format(**kwargs)

In [68]:
prompt = CustomPromptTemplate(input_variables=["input", "intermediate_steps"],
                              template=template,validate_template=False)

In [69]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish

        print(f"parse: {llm_output} !!!")
        
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
        
output_parser = CustomOutputParser()

In [70]:

os.environ["SERPER_API_KEY"] = '57f5529cc26567e265b100d9c20f58c0921452a8'

search = GoogleSerperAPIWrapper()
tool_google = Tool(
        name="Google Search",
        func=search.run,
        description="useful for when you need to ask with search"
    )

tool_names = [tool_google]

In [71]:
llm_chain = LLMChain(llm=llm, prompt=prompt)
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:", "\nObservations:"], 
    allowed_tools=tool_names
)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tool_names, verbose=True)

In [72]:
agent_executor.run("Is Eminem an actor?")



> Entering new AgentExecutor chain...
format. kwargs: {'input': 'Is Eminem an actor?', 'intermediate_steps': []} !!!!
parse: 
Question: Is Eminem an actor?
Thought: Let me see if there are any recent news or updates on this topic.
Action: Google Search
Action Input: "Eminem" + "actor" !!!

Question: Is Eminem an actor?
Thought: Let me see if there are any recent news or updates on this topic.
Action: Google Search
Action Input: "Eminem" + "actor"

Observation:Bodied
format. kwargs: {'input': 'Is Eminem an actor?', 'intermediate_steps': [(AgentAction(tool='Google Search', tool_input='Eminem" + "actor', log='\nQuestion: Is Eminem an actor?\nThought: Let me see if there are any recent news or updates on this topic.\nAction: Google Search\nAction Input: "Eminem" + "actor"'), 'Bodied')]} !!!!
format. text_match: 
Question: Is Eminem an actor?
Thought: Let me see if there are any recent news or updates on this topic.
Action: Google Search
Action Input: "Eminem" + "actor"  !!!!
match: Let m

'Yes, Eminem is an actor, having appeared in the movie Bodied.'

In [73]:
agent_executor.run("在中国，酒驾撞人会受到什么惩罚？")



> Entering new AgentExecutor chain...
format. kwargs: {'input': '在中国，酒驾撞人会受到什么惩罚？', 'intermediate_steps': []} !!!!
parse: Question: What are the consequences of drunk driving in China?
Thought: This is a serious issue and needs immediate attention.
Action: Google Search
Action Input: What are the consequences of drunk driving in China? !!!
Question: What are the consequences of drunk driving in China?
Thought: This is a serious issue and needs immediate attention.
Action: Google Search
Action Input: What are the consequences of drunk driving in China?

Observation:Drunk drivers will have their driving license suspended, serving a 5-year ban before they can apply for a new license, and they will be jailed and/or fined after being tried in court. If you have China traffic tickets more than 12 points, please contact us for details.
format. kwargs: {'input': '在中国，酒驾撞人会受到什么惩罚？', 'intermediate_steps': [(AgentAction(tool='Google Search', tool_input='What are the consequences of drunk drivin

"In China, the consequences of drunk driving include losing your driver's license for five years, possible imprisonment or fines, and seeking legal help for multiple offenses."